In [14]:
import os
from pyspark.sql import SparkSession
import pandas as pd

# Caminho Python
# Aqui tive que definir o python que estava usando na minha maquina
PYTHON_PATH = r"C:\Users\Moises Drummer\anaconda3\python.exe"

# Passando variáveis spark
os.environ["PYSPARK_PYTHON"] = PYTHON_PATH
os.environ["PYSPARK_DRIVER_PYTHON"] = PYTHON_PATH
os.environ["HADOOP_OPTIONAL_TOOLS"] = "true"


In [15]:
# Aqui selecionei um diretório local:
file_path = "C:/Users/Moises Drummer/repo/beer_case/docker/jupyter/data/breweries.json"

# Crio as configurações do Spark Session
spark = SparkSession.builder \
        .master("local[1]") \
        .config("spark.driver.memory", "1g") \
        .config("spark.local.dir", "C:/temp/spark-temp") \
        .config("spark.hadoop.hadoop.native.io", "false") \
        .appName("Processed_layer") \
        .getOrCreate()
# Carrega o json para um dataFrame
silver_df = pd.read_json(file_path)
silver_c = pd.read_json(file_path)

spark_df = spark.createDataFrame(silver_df)
# Mostro as 5 primeiras linhas
print(silver_df.head())

                                     id                     name brewery_type  \
0  5128df48-79fc-4f0f-8b52-d06be54d0cec         (405) Brewing Co        micro   
1  9c5a66c8-cc13-416f-a5d9-0a769c87d318         (512) Brewing Co        micro   
2  34e8c68b-6146-453f-a4b9-1f6cd99a5ada  1 of Us Brewing Company        micro   
3  6d14b220-8926-4521-8d19-b98a2d6ec3db     10 Barrel Brewing Co        large   
4  e2e78bd8-80ff-4a61-a65c-3bfbd9d76ce2     10 Barrel Brewing Co        large   

                     address_1 address_2 address_3            city  \
0               1716 Topeka St      None      None          Norman   
1        407 Radam Ln Ste F200      None      None          Austin   
2          8100 Washington Ave      None      None  Mount Pleasant   
3                62970 18th St      None      None            Bend   
4  1135 NW Galveston Ave Ste B      None      None            Bend   

  state_province postal_code        country   longitude   latitude  \
0       Oklahoma  7306

In [16]:
## Crio um dataframe apenas para verificar campos que contém muitos nulos
verifica_null = silver_c.isnull().sum()
verifica_null.head()

id                 0
name               0
brewery_type       0
address_1        757
address_2       8260
dtype: int64

In [19]:
# Faço um tratamento em alguns campos

silver_df = spark_df.select(
        col("id"),
        col("name"),
        col("brewery_type"),
        col("street"),
        col("city"),
        col("state"),
        col("postal_code"),
        col("country"),
        col("longitude"),
        col("latitude"),
        col("phone"),
        col("website_url"),
        col("address_2"),
        col("address_3")
    )\
    .withColumn("address_2", when(col("address_2").isNull(), "N/A").otherwise(col("address_2")))

In [20]:
# Converto para Pandas e escrever arquivos parquet em um repositório local.
silver_output_path = "file:///C:/Users/Moises Drummer/repo/beer_case/docker/minio/data1/silver-layer"
silver_layer = silver_df.toPandas()
silver_layer.to_parquet(silver_output_path.replace("file:///", ""), partition_cols=["country"])